In [ ]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

import geopandas as gpd

from routingpy.routers import MapboxOSRM

from shapely.geometry import Polygon
import shapely.geometry
import shapely.wkt
from shapely.wkt import loads
from shapely.validation import make_valid

import matplotlib.pyplot as plt

from palettable.colorbrewer.sequential import Reds_9

from lonboard import viz

In [ ]:
#read in our csv of Dallas Trauma Centers
df = pd.read_csv('data/raw/dallas_hospitals.csv')

#convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat'], crs='EPSG:4326'))
gdf.explore(tiles='CartoDBDarkMatter')

In [4]:
#Read txt file with our API key
with open('mbx_api_key.txt') as f:
    api_key = f.read()

In [5]:
#This was mostly borrowed (*cough*stolen*cough*) from Kyle Walker here: https://walker-data.com/posts/python-isochrones/

#Get API key we just read in from our text file
mb = MapboxOSRM(api_key = api_key)

def mb_isochrone(gdf: gpd.GeoDataFrame, 
                 time = [5, 10, 15], #must be either 5, 10 or 15 minutes
                 profile = "driving-traffic"): #use driving with traffic profile by default

    #get our lat/long coordinates as a list of lists
    coordinates = gdf[['long', 'lat']].values.tolist()

    # Create an empty list to store the isochrone geometries
    isochrone_shapes = []

    #create a list for the size of the isochrone we want to generate, expressed in minutes of time
    if type(time) is not list:
        time = [time]

    # API requires seconds so multiplify by 60
    time_seconds = [60 * x for x in time]

    # Given the way that routingpy works, we need to iterate through the list of 
    # coordinate pairs, then iterate through the object returned and extract the 
    # isochrone geometries.  
    for c in coordinates:
        iso_request = mb.isochrones(locations = c, #lat/long pair
                                    profile = profile, #driving with traffic
                                    intervals = time_seconds, #the time period, expressed in seconds
                                    polygons = "true") #return polygons
        
        #returns the polygons for the isochrone
        for i in iso_request:
            iso_geom = Polygon(i.geometry[0])
            isochrone_shapes.append(iso_geom)

    # Rebuild the dataset but with isochrone geometries
    df_values = gdf.drop(columns = ['geometry', 'long', 'lat'])

    time_col = time * len(df_values)

    # We'll need to repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis = 0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gpd.GeoDataFrame(
        data = df_values_rep,
        geometry = isochrone_shapes,
        crs = 4326
    )

    isochrone_gdf['time'] = time_col

    # We are sorting the dataframe in descending order of time to improve visualization
    # (the smallest isochrones should go on top, which means they are plotted last)
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending = False)

    return(isochrone_gdf)

In [ ]:
#create empty GeoDataFrame to store isochrones in
gdfx2 = gpd.GeoDataFrame()

#add isochrones iterativley to gdf
for i in [5, 10, 15]:
    gdfx = mb_isochrone(gdf, time = i)
    gdfx2 = pd.concat([gdfx, gdfx2])

gdfx2 = gdfx2.reset_index(drop=True).copy()

gdfx2.explore('time', cmap='Reds', tiles='CartoDBDarkMatter')

In [ ]:
gdfx2.loc[((gdfx2['aha_id'] == 6741050) & (gdfx2['time'] == 15))].explore(tiles='CartoDBDarkMatter')

In [8]:
#create a new column based on how far many minutes away the isochrone is ie. geometry_5, geometry_10 etc.
for i in list(gdfx2['time'].unique()):
    gdfx2.loc[gdfx2['time'] == i, f'geometry_{i}'] =  gdfx2['geometry']

#reset index
gdfx2 = gdfx2.reset_index(drop=True).copy()

#make a copy of the original gdf
gdfx3 = gdfx2.copy()

In [9]:
#Back to my code — loop through the isochrones for each trauma center
for center in list(gdfx3['tcn'].unique()):
    #loop through each time period for that trauma centers' isochrones, in increments of 3 mins from 3 to 27 mins.
    for i in range(5, 15, 5): 
        #take the area covered by the smaller isochrone and remove it from the next biggest isochrone to avoid overlap, ie. 6min - 3min
        big  = gdfx3.loc[((gdfx3['tcn'] == center) & (gdfx3[f'geometry_{i+5}'].notna())), f'geometry_{i + 5}'].iloc[0]
        small = gdfx3.loc[((gdfx3['tcn'] == center) & (gdfx3[f'geometry_{i}'].notna())), f'geometry_{i}'].iloc[0]
        gdfx3.loc[((gdfx3['tcn'] == center) & (gdfx3['time'] == (i + 5))), 'geometry'] = big.difference(small)


#make a copy of the output, in case we want to re-run and start from here.
gdfx4 = gdfx3.reset_index(drop=True).copy()

#now delete those columns we made earlier for each time period.
for i in range(5, 20, 5):
    gdfx4 = gdfx4.drop([f'geometry_{i}'], axis=1)

In [ ]:
gdfx4.loc[((gdfx4['aha_id'] == 6741050) & (gdfx2['time'] == 15))].explore(tiles='CartoDBDarkMatter')

In [11]:
#First trace the boundaries for all the isochrone polygon intersections
lines = gdfx4.apply(lambda x: x["geometry"].boundary, axis=1).to_list() #Create a list of all polygon boundaries, both outer and inner rings

# Combine all of those lines into a big multiline shape
noded_lines = shapely.ops.unary_union(lines) 

#Convert that big multiline shape back into component polygons
noded_lines_singleparts = [x for x in noded_lines.geoms] #First convert to a list of LineStrings
new_polys = list(shapely.polygonize(noded_lines_singleparts).geoms) #Then make polygons

In [12]:
#Transfer attributes from original polygons to newly formed polygons, by using points in the polygons

#Create point-in-polygon features by creating a dataframe from the list of polygons  
new_polys = gpd.GeoDataFrame(geometry=new_polys, crs=gdfx4.crs) 

#Create a copy of it to use to create points in polygons
new_polys_point = new_polys.copy() 
new_polys_point["geometry"] = new_polys.geometry.representative_point()

#Overlay points with original polygons, transferring the attributes (ie. minutes from trauma center)
points_with_attributes = gpd.sjoin(left_df=new_polys_point, right_df=gdfx4, how="left", predicate="within")

#Join the points to the newly formed polygons, transferring the attributes (minutes from trauma center):
del(points_with_attributes["index_right"]) #delete index_right col to make join wont fail

new_polys_with_attributes = gpd.sjoin(left_df=new_polys, right_df=points_with_attributes,
                                      how="left", predicate="intersects")

In [13]:
# Sort the polygon table by geometry and minutes from the trauma center.
new_polys_with_attributes["wkt"] = new_polys_with_attributes.apply(lambda x: x['geometry'].wkt, axis=1)
new_polys_with_attributes = new_polys_with_attributes.sort_values(by=["wkt", "time"], ascending=[True, False])
new_polys_with_attributes = new_polys_with_attributes.drop_duplicates(subset="wkt", keep="last")

In [14]:
#filter to only columns we need
gdfx5 = new_polys_with_attributes[['tcn', 'time', 'geometry']]

In [ ]:
gdfx5.explore('time', cmap='Reds', tiles='CartoDBDarkMatter')

In [16]:
#export to most optimized format
gdfx5.to_parquet('data/created/bleeding_out_nicar_demo_isochrones.parquet')